In [ ]:
## IMPORTS

import os
import sys
import datetime
import subprocess
import numpy as np
import tensorflow as tf
from osgeo import gdal
from osgeo import ogr
import matplotlib.pyplot as plt

In [ ]:
## SETTINGS

# inputs
area = "BER"
dinp = "D:\\uni\\deep" # dir input
fimg = os.path.join(dinp, "image.tif") # image file
fshp = os.path.join(dinp, "hymap_lcz_test.shp") # shp file
attr = "classNum" # class attribute in shapefile
dtbo = "C:\\Users\\Gekko\\Anaconda3\\Scripts\\tensorboard" # dir tb
targ = os.path.join(dinp, "subset2.tif") # tmp: image subset for classification

# outputs
dptr = os.path.join(dinp, "img_patches", "train") # dir image patches train
dpte = os.path.join(dinp, "img_patches", "test") # dir image patches test
fmod = os.path.join(dinp, "tf_model", "TFmodel.ckpt") # output model
dtbl = os.path.join(dinp, "tb_logs") # tensorboard dir logs

# network settings
sampleDistance = 6
trainTestRatio = 0.8
maxPatchesPerClass = 25
patchSize = 32
numChannels = 5
numClasses = 8

learning_rate = 0.001
epochs = 100
batch_size = 16
dropout = 0.75
batchsize = 8
display_step = 5

In [ ]:
## TENSORBOARD

subprocess.Popen([dtbo, "--logdir=testor:" + dtbl, "--port=6006"])
os.startfile(str("http://" + os.environ['COMPUTERNAME'] + ":6006/#"))

In [ ]:
## PRODUCE IMAGE PATCHES

# create outputfolder
if not os.path.exists(dptr):
    os.makedirs(dptr)
    os.makedirs(dpte)

# import image
img = gdal.Open(fimg)

# rasterize shapefile
shp = ogr.Open(fshp)
shpl = shp.GetLayer()
rs = gdal.GetDriverByName('GTiff')
shpr = fshp[:-4]+".tif"
rs = rs.Create(shpr, img.RasterXSize, img.RasterYSize, 1, gdal.GDT_Byte)
rs.SetGeoTransform(img.GetGeoTransform())
rs.SetProjection(img.GetProjectionRef())
gdal.RasterizeLayer(rs, [1], shpl, options=[str("ATTRIBUTE=" + attr)])
rs = None

# import rasterized shapefile
spl = gdal.Open(shpr)
splval= spl.ReadAsArray()

# create image patches
def savePatch(cla, idx, loc, path):
    patchFileName = os.path.join(path, str(cla) + "_" + area + "_" + \
                                 str('{0:0'+str(len(str(maxPatchesPerClass)))+'d}').format(idx+1) + ".tif")
    gdal.Translate(patchFileName, fimg, format = 'GTiff', srcWin = [loc[1]-patchSize/2,
                                                                    loc[0]-patchSize/2,
                                                                    patchSize,
                                                                    patchSize])

disGrid = np.zeros((splval.shape[0], splval.shape[1]))
disGrid[::sampleDistance, ::sampleDistance] = 1

pixelsNum = np.zeros((numClasses))
patchesNum = np.zeros((numClasses))

for cla in np.unique(splval):
    if cla > 0 and cla < 99:
        xy = np.argwhere(splval == cla)
        xy = xy[np.random.permutation(xy.shape[0])]
        pixelsNum[cla-1] = xy.shape[0]
        
        xyList = []
        maxCounter = 0
        for loc in xy:
            if disGrid[loc[0], loc[1]] == 1 and maxCounter < maxPatchesPerClass:
                maxCounter += 1
                patchesNum[cla-1] += 1
                xyList.append(loc)
        
        split = int(patchesNum[cla-1] * trainTestRatio)
        patchesTrain = xyList[:split]
        patchesTest= xyList[split:]
        
        for idx, loc in enumerate(patchesTrain):
            savePatch(cla, idx, loc, dptr)
        for idx, loc in enumerate(patchesTest):
            savePatch(cla, idx, loc, dpte)
            
print("pixel per class: ", pixelsNum)
print("patTr per class: ", patchesNum * trainTestRatio)
print("patTe per class: ", patchesNum * (1-trainTestRatio))

In [ ]:
## NORMALIZE

pTrain = os.listdir(dptr)
pTest = os.listdir(dpte)

patchesX = []
for idx, f in enumerate(pTrain):
    patchval = gdal.Open(os.path.join(dptr, f)).ReadAsArray()
    patchval = np.moveaxis(patchval, 0, -1)
    patchesX.append(patchval)
for idx, f in enumerate(pTest):
    patchval = gdal.Open(os.path.join(dpte, f)).ReadAsArray()
    patchval = np.moveaxis(patchval, 0, -1)
    patchesX.append(patchval)

means = []
stds = []
z = np.array(patchesX, dtype=np.float32)
for i in range(z.shape[3]):
    means.append(np.mean(z[:, :, :, i]))
    stds.append(np.std(z[:, :, :, i]) )
    
np.savetxt(os.path.join(dinp, "means.txt"), means)
np.savetxt(os.path.join(dinp, "stds.txt"), stds)

In [ ]:
## TF MODEL SETUP

X = tf.placeholder(tf.float32, [None, patchSize, patchSize, numChannels])
Y = tf.placeholder(tf.float32, [None, numClasses])
keep_prob = tf.placeholder(tf.float32)
phaseTrain = tf.placeholder(tf.bool)

def conv2d(x, W, b, strides=1, phase=0):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    x = tf.layers.batch_normalization(x, training=phase)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def conv_net(x, weights, biases, dropout, phaseTrain):
    conv1 = conv2d(x, weights['wc1'], biases['bc1'], phase=phaseTrain)
    conv1 = maxpool2d(conv1, k=2)
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'], phase=phaseTrain)
    conv2 = maxpool2d(conv2, k=2)
    
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, numChannels, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([4096, 128])),
    'out': tf.Variable(tf.random_normal([128, numClasses]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([128])),
    'out': tf.Variable(tf.random_normal([numClasses]))
}

logits = conv_net(X, weights, biases, keep_prob, phaseTrain)
prediction = tf.nn.softmax(logits)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
## TRAINING

# tensorboard
tf.summary.image('input_x', X[:12, :, :, :3], 12) 
tf.summary.scalar('accuracy', accuracy)
tf.summary.scalar('loss', loss_op)
merge = tf.summary.merge_all()
time = datetime.datetime.now().strftime("%Y%m%d_%H_%M")
train_writer = tf.summary.FileWriter(os.path.join(dtbl, "train", time))
test_writer = tf.summary.FileWriter(os.path.join(dtbl, "test", time))

# import statistics
means = np.loadtxt(os.path.join(dinp, "means.txt"))
stds = np.loadtxt(os.path.join(dinp, "stds.txt"))

# import test patches
pTest = os.listdir(dpte)
X_test = []
Y_test = []
for f in pTest:
    pclass = int(f.split("_")[0])
    labtmp = np.zeros((numClasses))
    labtmp[pclass-1] = 1
    Y_test.append(labtmp)
    
    patchval = gdal.Open(os.path.join(dpte, f)).ReadAsArray()
    patchval = np.array(patchval, dtype=np.float32)
    for i in range(patchval.shape[0]):
        patchval[i, :, :] = (patchval[i, :, :] - means[i]) / stds[i]
    patchval = np.moveaxis(patchval, 0, -1)
    X_test.append(patchval)

# calc batch sizes
pTrain = os.listdir(dptr)
pTrain = [pTrain[i] for i in np.random.permutation(len(pTrain))]

batches = []
for i in range(0, len(pTrain), batchsize):
    batches.append(i)
if not batches[-1] == len(pTrain):
    batches.append(len(pTrain))

def getTrainBatch(b):
    X_train = []
    Y_train = []
    for f in pTrain[batches[b]:batches[b+1]]:
        pclass = int(f.split("_")[0])
        labtmp = np.zeros((numClasses))
        labtmp[pclass-1] = 1

        patchval = gdal.Open(os.path.join(dptr, f)).ReadAsArray()
        patchval = np.array(patchval, dtype=np.float32)
        for i in range(patchval.shape[0]):
            patchval[i, :, :] = (patchval[i, :, :] - means[i]) / stds[i]
        
        # augmentation
        for r in range(0,4):
            # rotate
            patchValRotated = np.rot90(patchval, k=r, axes=(1, 2))
            X_train.append(np.moveaxis(patchValRotated, 0, -1))
            Y_train.append(labtmp)
            # flip rotated 
            patchValRotated = np.fliplr(patchValRotated)
            X_train.append(np.moveaxis(patchValRotated, 0, -1))
            Y_train.append(labtmp)

    return [X_train, Y_train]
    
# start training
saver = tf.train.Saver()
print("Epoch".ljust(len(str(epochs))*2+4) + "Train   Test    MBLoss")

with tf.Session() as sess:  
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(1, epochs+1): 
        for b in range(len(batches)-1):
            
            X_train, Y_train = getTrainBatch(b)
            
            sess.run(train_op, feed_dict={X: X_train, Y: Y_train, keep_prob: dropout, phaseTrain: 1})

        if epoch % display_step == 0 or epoch == 1:
            summaryTr, lossTr, accTr = sess.run([merge, loss_op, accuracy],
                                                feed_dict={X: X_train,
                                                           Y: Y_train,
                                                           keep_prob: 1.0,
                                                           phaseTrain: 0})
            
            summaryTe, accTe = sess.run([merge, accuracy], feed_dict={X: X_test,
                                                                      Y: Y_test,
                                                                      keep_prob: 1.0,
                                                                      phaseTrain: 0})

            print(str('{0:0'+str(len(str(epochs)))+'d}').format(epoch) + "/" + str(epochs) + \
                  "   {:.3f}".format(accTr) + \
                  "   {:.3f}".format(accTe) + \
                  "   {:.3f}".format(lossTr))

            train_writer.add_summary(summaryTr, epoch)
            test_writer.add_summary(summaryTe, epoch)

            save_path = saver.save(sess, fmod)

In [ ]:
## VALIDATION

# import statistics
means = np.loadtxt(os.path.join(dinp, "means.txt"))
stds = np.loadtxt(os.path.join(dinp, "stds.txt"))

# import test patches
pTest = os.listdir(dpte)
X_test = []
Y_test = []
Y_ref = np.zeros((len(pTest))).astype(int)

for idx, f in enumerate(pTest):
    pclass = int(f.split("_")[0])    
    labtmp = np.zeros((numClasses))
    labtmp[pclass-1] = 1
    Y_test.append(labtmp)
    
    Y_ref[idx] = pclass
    
    patchval = gdal.Open(os.path.join(dpte, f)).ReadAsArray()
    patchval = np.array(patchval, dtype=np.float32)
    for i in range(patchval.shape[0]):
        patchval[i, :, :] = (patchval[i, :, :] - means[i]) / stds[i]
    patchval = np.moveaxis(patchval, 0, -1)
    X_test.append(patchval)

# predict
with tf.Session() as sess:  
    tf.train.Saver().restore(sess, fmod)
    
    Y_pred = sess.run(prediction, feed_dict={X: X_test, Y: Y_test, keep_prob:1.0, phaseTrain: 0})
    Y_pred = np.argmax(Y_pred, 1) + 1

# accuracy matrix
accMat = np.zeros((numClasses, numClasses))
for i in range(Y_ref.shape[0]):
    accMat[Y_pred[i]-1, Y_ref[i]-1] += 1
print(accMat)

# accuracies
print("UA", np.around(np.diagonal(accMat) / np.sum(accMat, 0), 2))
print("PA", np.around(np.diagonal(accMat) / np.sum(accMat, 1), 2))
print("OA", np.around(np.sum(np.diagonal(accMat)) / np.sum(accMat) * 100, 2))

In [ ]:
## CLASSIFICATION

# import
imgc = gdal.Open(targ)
img = imgc.ReadAsArray().astype(np.float32)

# normalize
for i in range(len(means)):
    img[i, :, :] = (img[i, :, :] - means[i]) / stds[i]

# classify
cmap = np.zeros((img.shape[1], img.shape[2]))
Ypatches = np.zeros((img.shape[2]-patchSize, numClasses))

with tf.Session() as sess:
    tf.train.Saver().restore(sess, fmod)
    
    for x in range(0, img.shape[1]-patchSize):
        Xpatches = []
        for y in range(0, img.shape[2]-patchSize):
            Xpatch = img[:, x:(x+patchSize), y:(y+patchSize)]
            Xpatch = np.moveaxis(Xpatch, 0, -1)
            Xpatches.append(Xpatch)
            
        pred = sess.run(prediction, feed_dict={X: Xpatches, Y: Ypatches, keep_prob:1.0, phaseTrain: 0})
        cmap[x+int(patchSize/2), int(patchSize/2):int(patchSize/2)+pred.shape[0]] = np.argmax(pred, 1) + 1

        sys.stdout.write('\rProgress ' + str("{0:.2f}".format(round(x / (img.shape[1]-patchSize)*100, 2))) + "%")
        
# plot
plt.figure(figsize=(20,10))
imgplot = plt.imshow(cmap)

# export
driver = gdal.GetDriverByName('GTiff')
df = driver.Create(targ[:-4]+"_deep.tif", imgc.RasterXSize, imgc.RasterYSize, 1, gdal.GDT_Byte)
df.SetGeoTransform(imgc.GetGeoTransform())
df.SetProjection(imgc.GetProjectionRef())
df.GetRasterBand(1).WriteArray(cmap)
df.FlushCache()
df = None

In [ ]:
## OLOFFSON ACC

np.set_printoptions(suppress=True)
e = accMat
c, a = np.unique(cmap, return_counts=True)
a = a[1:]

img = gdal.Open(fimg)
gt = img.GetGeoTransform()
areaFac = gt[1] * -gt[5] / 10000

p = (1.0 * e.transpose() * a / np.sum(a) / np.sum(e, 1)).transpose()

print("AREA PROD")
PArea = areaFac * np.sum(p, 0) * np.sum(a)
PAreaCI = areaFac * 1.96 * np.sum(a) * np.sqrt(np.sum((((((((1. * e.transpose() / np.sum(e, 1)).transpose()) * (1 - ((1. * e.transpose() / np.sum(e, 1)).transpose()))).transpose() / (np.sum(e, 1) - 1)).transpose()).transpose() * np.power(np.sum(p, 1), 2)).transpose()), 0))
print(np.round(PArea, 2))
print(np.round(PAreaCI, 2))

print("UA")
ua = (np.diagonal(p).transpose() / np.sum(p, 1)).transpose()
uaCI = 1.96 * np.sqrt(ua * (1 - ua) / (np.sum(e, 1) - 1))
print(np.around(ua, 2))
print(np.around(uaCI, 2))

print("PA")
pa = np.diagonal(p) / np.sum(p, 0)
N = []
X = []
for i in range(0, len(a)):
    N.append(np.sum(a / np.sum(e, 1) * e[:, i]))
    xa = np.array([x for j, x in enumerate(a) if j != i])
    xb = np.array(e[[x for j, x in enumerate(np.arange(len(a))) if j != i], i])
    xc = np.array([x for j, x in enumerate(np.sum(e, 1)) if j != i])
    X.append(np.sum(xa ** 2 * xb / xc * (1 - xb / xc) / (xc - 1)))
paCI = 1.96 * np.sqrt(1. / (np.array(N) ** 2) * (a ** 2 * (1 - pa) ** 2 * ua * (1 - ua) / (np.sum(e, 1) - 1) + pa ** 2 * X))
print(np.around(pa, 2))
print(np.around(paCI, 2))

print("OA")
oa = np.sum(np.diagonal(p))
oaCI = 1.96 * np.sqrt(np.sum(np.sum(p, 1) * np.sum(p, 1) * ua * (1 - ua) / (np.sum(e, 1) - 1)))
print(np.around(oa, 2))
print(np.around(oaCI, 2))
